# **DASA CS-2: [TAS: Tele Assistance System]**

In [1]:
# -*- coding: utf-8 -*-
# Native imports
import os
import re
import sys
import time

# Third-party imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# PyDASA imports
from Src.PyDASA.Measure.fdu import FDU
from Src.PyDASA.Measure.params import Parameter, Variable
from Src.PyDASA.Utils.cstm import RegexManager
import Src.PyDASA.Utils.cfg as PyDASAcfg
from Src.PyDASA.Model.dim import DimensionalModel, DimensionalAnalyzer
from Src.PyDASA.Simul.sens import Sensitivity
from Src.PyDASA.Simul.rprt import SensitivityAnalysis
# from Src.PyDASA.Simul.moca import MonteCarloSimulation

## **Project Overview**

- TAS offers telehealth services to patients at home.
- It uses third-party services for medical analysis, emergency alarms, and drug delivery.
- Self-adaptation deals with uncertainty (changing service quality, patient needs).
- The system uses runtime models, simulations, and a MAPE-K feedback loop (ActivFORMS runtime environment) to adapt proactively​

## **Software Architecture Summary**

### **Base Design**
Service-oriented architecture (SOA) + MAPE-K feedback loop
- **Service-oriented architecture (SOA):** Each Comp is a service that can be replaced or reconfigured.
- **MAPE-K feedback loop:** Monitors the system, analyzes its state, plans adaptations, and executes them based on knowledge stored in the Knowledge Base.

Dividing TAS into Target and Controller Parts (MAPE-K Structure)

| Subsystem | Comps |
|:---|:---|
| **Target System (Managed System)** | - **TAS Core Services:** Vital Parameters Monitor, Medication Management, Emergency Response Trigger <br> - **Service Integrations:** Medical Analysis Services (MAS), Alarm Services (AS), Drug Services (DS) from third-party providers <br> - **Probes:** Instruments to collect data about vital signs, service invocations, failures, and latencies <br> - **Effectors:** Mechanisms to replace or reconfigure services dynamically |
| **Controller (Managing System)** | - **Monitor:** Observes system state, updates runtime models (service failures, delays, costs) |
| | - **Analyzer:** Runs simulations (runtime statistical model checking) on updated models to predict system qualities |
| | - **Planner:** Selects best service configuration based on goals (availability, performance, cost) |
| | - **Executor:** Applies adaptations (switches services, reconfigures workflows) |
| | - **Knowledge Base:** Maintains models of the system, environment models, adaptation goals, quality models |

Reference: Based on MAPE-K division and ActivFORMS architecture.

### **Notes**
- TAS (Tele Assistance System) operates in a dynamic environment where service quality, availability, and user needs frequently change.
- It implements **self-adaptive mechanisms** using the MAPE-K feedback loop.
- Adaptation is critical to maintaining **reliability**, **performance**, and **compliance** with patient care standards.
- ActivFORMS provides the runtime framework for model-based adaptation using runtime models, simulations, and verified decision-making.
---

### **Functional Requirements (Reported Capabilities)**

| ID | Description |
|:-----|:---|
| RC-001 | Provide remote health support for patients with chronic conditions within their homes. |
| RC-002 | Periodically measure patients' vital signs using wearable devices and sensors. |
| RC-003 | Analyze patient data using third-party medical analysis services to assess health status. |
| RC-004 | Invoke pharmacy services to deliver medication if analysis indicates a treatment change is needed. |
| RC-005 | Adjust medication dosage dynamically based on updated health analysis. |
| RC-006 | Send automatic alarms and alerts to emergency services during critical situations. |
| RC-007 | Dynamically integrate new third-party services into existing workflows to enhance system functionality. |
| RC-008 | Adapt to changing healthcare goals by modifying workflows and service selections in real-time. |
| RC-009 | Ensure system reliability by switching to equivalent services when current ones fail or degrade. |
| RC-010 | Continuously monitor service performance and adjust workflows to optimize reliability, performance, and cost. |

---

### **Quality Scenarios (QS)**

| ID | Quality Attribute | Sub-Category | Scenario Description |
|:-----|:---|:---|:---|
| QS-001 | Availability | Fault Tolerance | TAS must detect third-party service failures and switch to a backup service within 30 seconds. |
| QS-002 | Performance | Response Time | TAS must process health analysis results and issue alerts within 2.5 seconds. |
| QS-003 | Interoperability | Syntactic/Semantic | TAS must successfully integrate new third-party services dynamically without downtime. |
| QS-004 | Usability | Learnability | Patients must access and understand health data via the TAS interface within 5 minutes of onboarding. |
| QS-005 | Security | Confidentiality | Patient data must be encrypted during transmission and storage, ensuring HIPAA compliance. |
| QS-006 | Adaptability | Dynamic Reconfiguration | TAS must reconfigure workflows dynamically based on real-time patient data and healthcare goals. |

---

### **Adaptation Scenarios (AS)**

| ID | Scenario | Description | Adaptation Strategy |
|:-------|:---|:---|:---|
| AS-001 | Service Failure | A third-party service fails to respond, impacting TAS operations. | Switch to an equivalent service; use redundancy (invoke multiple services simultaneously if needed). |
| AS-002 | Variation in Service Response Time | A third-party service exceeds expected response times, degrading TAS performance. | Switch to a more performant service; prioritize services based on real-time response metrics. |
| AS-003 | New Service Introduction | A new third-party service becomes available. | Dynamically recognize and integrate new services into the registry and workflows. |
| AS-004 | Changing Goals | Healthcare objectives or patient needs change over time. | Adjust workflows and select appropriate services to meet updated goals. |
| AS-005 | Wrong Operation Sequence | Errors or inefficiencies in workflow execution. | Redesign and reconfigure the workflow sequence to ensure compliance with functional requirements. |

---

#### **FDU (Fundamental Design Unit)**

In [2]:
# wotking dimensional framework
# DIGITAL has []
tas_fwk = "DIGITAL"
print(f"Working with '{tas_fwk}' dimensional framework.\n")
# cheking in config file
if tas_fwk not in PyDASAcfg.FDU_FWK_DT:
    print(f"Framework {tas_fwk} not found in config file")
print("=== FDU list ===")
for k, v in PyDASAcfg.DIGI_FDU_PREC_DT.items():
    print(f"\t'{k}' = '{v}'")
    
# setting up working DFUs
fdu_rm = RegexManager(_fwk=tas_fwk)
fdu_rm.update_global_regex()

Working with 'DIGITAL' dimensional framework.

=== FDU list ===
	'T' = 'Time'
	'D' = 'Data'
	'M' = 'Complexity'
	'K' = 'Conectivity'
	'C' = 'Capacity'


In [3]:
# checking if the framework changed in the cfg module
# custom regex for FDU
print("\n==== Custom Regex ====")
print("\tWKNG_DFLT_FDU_PREC_LT:", PyDASAcfg.WKNG_FDU_PREC_LT)
print("\tWKNG_FDU_REGEX:", PyDASAcfg.WKNG_FDU_REGEX)
print("\tWKNG_POW_REGEX:", PyDASAcfg.WKNG_POW_REGEX)
print("\tWKNG_NO_POW_REGEX:", PyDASAcfg.WKNG_NO_POW_REGEX)
print("\tWKNG_FDU_SYM_REGEX:", PyDASAcfg.WKNG_FDU_SYM_REGEX)


==== Custom Regex ====
	WKNG_DFLT_FDU_PREC_LT: ['T', 'D', 'M', 'K', 'C']
	WKNG_FDU_REGEX: ^[TDMKC](\^-?\d+)?(\*[TDMKC](?:\^-?\d+)?)*$
	WKNG_POW_REGEX: \-?\d+
	WKNG_NO_POW_REGEX: [TDMKC](?!\^)
	WKNG_FDU_SYM_REGEX: [TDMKC]


#### **Dimensional Variables**

In [4]:
# list of variables
print("\n==== Variables ====")
tas_1_lt = [
    Variable(_sym="l",    # TAS Comp 1 arrival rate
             _fwk=tas_fwk,
             _cat="INPUT",
             _units="req/s",
             _std_units="req/s",
             name="TAS1 Arrival Rate",
             _dims="M*T^-1",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=344.83,
             _std_max=344.83,
    ),
    Variable(_sym="N",       # TAS Comp 1 current request in queue
             _fwk=tas_fwk,
             _cat="INPUT",
             _units="req",
             _std_units="req",
             name="TAS1 Current Request in Queue",
             _dims="M",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=1,
             _std_max=1,
             ),
    Variable(_sym="W",         # TAS Comp 1 mean response time
             _fwk=tas_fwk,
             _cat="OUTPUT",
             _units="s/req",
             _std_units="s/req",
             name="TAS1 Mean Response Time",
             _dims="T*M^-1",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=2.02/100,
             _std_max=2.02/100,
             ),
    Variable(_sym="u",        # TAS Comp 1 service rate
             _fwk=tas_fwk,
             _cat="CONTROL",
             _units="req/s",
             _std_units="req/s",
             name="TAS1 Service Rate",
             _dims="M*T^-1",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=1000.00,
             _std_max=1000.00,
             ),
    Variable(_sym="x",       # TAS Comp 1 departure rate
             _fwk=tas_fwk,
             _cat="CONTROL",
             _units="req/s",
             _std_units="req/s",
             name="TAS1 Departure Rate",
             _dims="M*T^-1",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=344.83,
             _std_max=344.83,
             ),
    Variable(_sym="L",     # TAS Comp 1 mean number of requests
             _fwk=tas_fwk,
             _cat="CONTROL",
             _units="req",
             _std_units="req",
             name="TAS1 Mean Number of Requests in System",
             _dims="M",
             relevant=True,
             _min=0,
             _std_min=0,
             _max=2.333,
             _std_max=2.333,
             ),
]

"""
C Vs. Metric	λi [req/s]	Χi [req/s]	μi [req/s]
*C1	                344.83	344.83	1000.00
*C2	                258.62	258.62	1000.00
*C3	                174.14	174.14	1000.00
C4	                88.17	77.59	180.00
C5	                132.09	122.84	530.00
C6	                70.96	58.19	150.00
C7	                65.22	58.05	700.00
C8	                60.46	58.05	410.00
C9	                70.79	58.05	1580.00
*C10	            258.62	258.62	1000.00
C11	                179.67	183.62	550.00
*C12	            174.14	174.14	1000.00
*C13	            170.69	174.44	1000.00
Sum	                157.57	153.94	776.92

Avg for all C1-C13	λi [req/s]	Χi [req/s]	μi [req/s]
f(M/M/1/∞/FIFO)	f(M/M/1/∞/FIFO)
Li [req]	Wi [s*req^(-1)]
2.333E+00	2.020E-02


"""

for i, var in enumerate(tas_1_lt):
    var.idx = i
    print(f"Variable {var.idx}: '{var._sym}', ({var._cat})")
    print(f"\tStd Units: '{var._std_units}")
    print(f"\tDimensionality: '{var._dims}'")
    


==== Variables ====
Variable 0: 'l', (INPUT)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
Variable 1: 'N', (INPUT)
	Std Units: 'req
	Dimensionality: 'M'
Variable 2: 'W', (OUTPUT)
	Std Units: 's/req
	Dimensionality: 'T*M^-1'
Variable 3: 'u', (CONTROL)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
Variable 4: 'x', (CONTROL)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
Variable 5: 'L', (CONTROL)
	Std Units: 'req
	Dimensionality: 'M'


#### **Dimensionless Coefficients**

In [5]:
# setting up the model
tas_1_dm = DimensionalModel(_idx=0,
                            _fwk=tas_fwk,
                            _sym="DM_1",
                            name="TAS1 Model",
                            _param_lt=tas_1_lt,)
tas_1_dm.relevance_lt = tas_1_lt
for i, var in enumerate(tas_1_dm.param_lt):
    var.idx = i
    print(f"Variable {var.idx}: '{var._sym}', ({var._cat})")
    print(f"\tStd Units: '{var._std_units}")
    print(f"\tDimensionality: '{var.dims}'")
    print(f"\t_sym_exp: '{var._sym_exp}'")

Variable 0: 'l', (INPUT)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
	_sym_exp: 'T**(-1)* M**(1)'
Variable 1: 'N', (INPUT)
	Std Units: 'req
	Dimensionality: 'M'
	_sym_exp: 'M**(1)'
Variable 2: 'W', (OUTPUT)
	Std Units: 's/req
	Dimensionality: 'T*M^-1'
	_sym_exp: 'T**(1)* M**(-1)'
Variable 3: 'u', (CONTROL)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
	_sym_exp: 'T**(-1)* M**(1)'
Variable 4: 'x', (CONTROL)
	Std Units: 'req/s
	Dimensionality: 'M*T^-1'
	_sym_exp: 'T**(-1)* M**(1)'
Variable 5: 'L', (CONTROL)
	Std Units: 'req
	Dimensionality: 'M'
	_sym_exp: 'M**(1)'


In [21]:
# solving the model
tas_1_solver = DimensionalAnalyzer(_fwk=tas_fwk,
                                   _idx=0,
                                   io_fdu=tas_1_lt,
                                   _fdu_mp=tas_1_dm._fdu_mp,
                                   _param_lt=tas_1_dm.param_lt,
                                   _relevance_lt=tas_1_dm._relevance_lt,)
9# checking the relevance list
print("\n=== Relevance List ===")
tas_1_solver.relevance_lt = tas_1_dm._relevance_lt
# creating the matrix
tas_1_solver.create_matrix()
# solving the matrix
tas_1_solver.solve_matrix()

print("\n=== Dimensionless Coefficients ===")
for coef in tas_1_solver.pi_coef_lt:
    print(f"'{coef.name}': ({coef.cat})")
    print(f"{coef.sym} = {coef.pi_expr}")
    print(f"{coef.par_dims}\n")


=== Relevance List ===

=== Dimensionless Coefficients ===
'Pi-Coefficient No. 0': (COMPUTED)
\Pi_{0} = l*W
{'l': 1.0, 'W': 1.0}

'Pi-Coefficient No. 1': (COMPUTED)
\Pi_{1} = \frac{u}{l}
{'l': -1.0, 'u': 1.0}

'Pi-Coefficient No. 2': (COMPUTED)
\Pi_{2} = \frac{x}{l}
{'l': -1.0, 'x': 1.0}

'Pi-Coefficient No. 3': (COMPUTED)
\Pi_{3} = \frac{L}{N}
{'N': -1.0, 'L': 1.0}



#### **Sensitivity Analysis**

In [7]:
tas_1_sanys = SensitivityAnalysis(_idx=0,
                                  _sym="SA1",
                                  _fwk=tas_fwk,
                                   name="Sensitivity Analysis TAS 1",
                                  _relevance_lt=tas_1_dm.relevance_lt,
                                  _coefficient_lt=tas_1_solver.pi_coef_lt,)
tas_1_sanys.analyze_pi_sensitivity(category="NUMERIC")      #, cutoff="max")

In [16]:
for r in tas_1_sanys.report:
    print(f"\n=== {r.sym} ===")
    res = r.result
    names = res["names"]

    # Print the formatted report
    print("=== Sensitivity Analysis Report ===")
    title = f"{'Variable':10} {'S1':<15} "
    title += f"{'S1_conf':<15} {'ST':<15} {'ST_conf':<15}"
    print(title)
    print("-" * 70)

    for i, name in enumerate(names):
        txt = f"{name:<10} {res['S1'][i]:<15.6f} {res['S1_conf'][i]:<15.6f} "
        txt += f"{res['ST'][i]:<15.6f} {res['ST_conf'][i]:<15.6f}"
        print(txt)


=== \Pi_{0} ===
=== Sensitivity Analysis Report ===
Variable   S1              S1_conf         ST              ST_conf        
----------------------------------------------------------------------
W          0.427538        0.018554        0.571498        0.043847       
l          0.427434        0.013477        0.571615        0.040676       

=== \Pi_{1} ===
=== Sensitivity Analysis Report ===
Variable   S1              S1_conf         ST              ST_conf        
----------------------------------------------------------------------
l          0.148291        0.012476        0.886974        0.037117       
u          0.013060        0.015247        0.096307        0.038177       

=== \Pi_{2} ===
=== Sensitivity Analysis Report ===
Variable   S1              S1_conf         ST              ST_conf        
----------------------------------------------------------------------
l          0.168320        0.011924        0.887862        0.040322       
x          0.009283        0

#### **Monte Carlo Simulation**

### **Arternative Design No.1 (Horizontal Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

### **Arternative Design No.2 (Vertical Scaling)**

#### **Dimensional Variables**

#### **Dimensionless Coefficients**

#### **Sensitivity Analysis**

#### **Monte Carlo Simulation**

## **Conclusion**

## **Future Work**